# Shortcuts Validation Notebook

This notebook validates that all generated shortcuts are true shortest paths.

In [45]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
print(f"Project root: {PROJECT_ROOT}")

Project root: /home/kaveh/projects/shortcuts-generation


## 1. Load Data

In [46]:
# DISTRICT = "Somerset"
DISTRICT = "Burnaby"

# Load edges with costs
edges_df = pd.read_csv(PROJECT_ROOT / f"data/{DISTRICT}_driving_simplified_edges_with_h3.csv")
edges_df['cost'] = edges_df['length'] / edges_df['maxspeed'].replace(0, np.inf)
print(f"Loaded {len(edges_df)} edges")
edges_df.head()

Loaded 34965 edges


,length,maxspeed,geometry,highway,cost,incoming_cell,outgoing_cell,lca_res,id
0,98.503,30.0,LINESTRING (-122.92154693603516 49.27764129638...,tertiary_link,3.283433,644733695069033029,644733695069283890,8,0
1,53.933,30.0,LINESTRING (-122.92154693603516 49.27764129638...,tertiary,1.797767,644733695069345412,644733695069283890,9,1
2,684.050,50.0,LINESTRING (-122.92154693603516 49.27764129638...,tertiary,13.681000,644733695063774494,644733695069283890,7,2
3,35.488,30.0,LINESTRING (-122.92096710205078 49.27793121337...,tertiary,1.182933,644733695069337920,644733695069345412,10,3
4,53.933,30.0,LINESTRING (-122.92096710205078 49.27793121337...,tertiary,1.797767,644733695069283890,644733695069345412,9,4


In [47]:
# Load edge graph (connectivity)
graph_df = pd.read_csv(PROJECT_ROOT / f"data/{DISTRICT}_driving_edge_graph.csv")
print(f"Loaded {len(graph_df)} edge connections")
graph_df.head()

Loaded 98901 edge connections


,incoming_edge,outgoing_edge
0,30721,30720
1,20799,1302
2,8066,8061
3,15745,15742
4,23542,23545


In [48]:
# Load generated shortcuts
shortcuts_df = pd.read_parquet(PROJECT_ROOT / f"output/{DISTRICT}_shortcuts_hybrid")
print(f"Loaded {len(shortcuts_df)} shortcuts")
shortcuts_df.head()

Loaded 4136992 shortcuts


,incoming_edge,outgoing_edge,cost,via_edge,inside,cell
0,0,10915,42.042058,14643,-2,613208497678450687
1,0,14643,10.043667,24588,1,604201298467749887
2,0,14644,10.043667,24588,-1,613208497678450687
3,1,0,3.595533,4,1,613208497678450687
4,2,1809,35.102200,9294,-2,613208497678450687


## 2. Build Full Edge Graph

In [49]:
# Create edge cost lookup
edge_costs = edges_df.set_index('id')['cost'].to_dict()

# Add costs to graph edges (cost of incoming_edge)
graph_df['cost'] = graph_df['incoming_edge'].map(edge_costs)

# Get all unique edges
all_edges = pd.concat([graph_df['incoming_edge'], graph_df['outgoing_edge']]).unique()
n_edges = len(all_edges)
edge_to_idx = {e: i for i, e in enumerate(all_edges)}

print(f"Total unique edges: {n_edges}")

Total unique edges: 34965


In [50]:
# Build sparse graph matrix
src_indices = graph_df['incoming_edge'].map(edge_to_idx).values
dst_indices = graph_df['outgoing_edge'].map(edge_to_idx).values
costs = graph_df['cost'].values

graph_matrix = csr_matrix((costs, (src_indices, dst_indices)), shape=(n_edges, n_edges))
print(f"Graph matrix shape: {graph_matrix.shape}")
print(f"Non-zero entries: {graph_matrix.nnz}")

Graph matrix shape: (34965, 34965)
Non-zero entries: 98901


## 3. Compute True Shortest Paths

In [51]:
print("Computing all-pairs shortest paths (this may take a while)...")
dist_matrix = shortest_path(csgraph=graph_matrix, method='auto', directed=True)
print(f"Done! Matrix shape: {dist_matrix.shape}")

Computing all-pairs shortest paths (this may take a while)...


Done! Matrix shape: (34965, 34965)


## 4. Validate Shortcuts

In [52]:
# Get indices for shortcuts
shortcuts_df['src_idx'] = shortcuts_df['incoming_edge'].map(edge_to_idx)
shortcuts_df['dst_idx'] = shortcuts_df['outgoing_edge'].map(edge_to_idx)

# Get true shortest path costs
shortcuts_df['true_cost'] = [
    dist_matrix[s, d] 
    for s, d in zip(shortcuts_df['src_idx'], shortcuts_df['dst_idx'])
]

# Compare with shortcut costs
shortcuts_df['diff'] = np.abs(shortcuts_df['cost'] - shortcuts_df['true_cost'])
shortcuts_df['is_optimal'] = shortcuts_df['diff'] < 1e-6

print(f"Total shortcuts: {len(shortcuts_df)}")
print(f"Optimal shortcuts: {shortcuts_df['is_optimal'].sum()}")
print(f"Non-optimal shortcuts: {(~shortcuts_df['is_optimal']).sum()}")

Total shortcuts: 4136992
Optimal shortcuts: 4136992
Non-optimal shortcuts: 0


In [53]:
# Show non-optimal shortcuts if any
non_optimal = shortcuts_df[~shortcuts_df['is_optimal']]
if len(non_optimal) > 0:
    print("\n❌ Non-optimal shortcuts found:")
    print(non_optimal[['incoming_edge', 'outgoing_edge', 'cost', 'true_cost', 'diff']].head(20))
else:
    print("\n✅ All shortcuts are optimal shortest paths!")


✅ All shortcuts are optimal shortest paths!


In [54]:
# Summary statistics
print("\n=== Validation Summary ===")
print(f"Total shortcuts: {len(shortcuts_df):,}")
print(f"Optimal: {shortcuts_df['is_optimal'].sum():,} ({100*shortcuts_df['is_optimal'].mean():.2f}%)")
print(f"Max cost difference: {shortcuts_df['diff'].max():.6f}")
print(f"Mean cost difference: {shortcuts_df['diff'].mean():.6f}")


=== Validation Summary ===
Total shortcuts: 4,136,992
Optimal: 4,136,992 (100.00%)
Max cost difference: 0.000000
Mean cost difference: 0.000000


## 5. Analyze Non-Optimal Shortcuts (if any)

In [55]:
if len(non_optimal) > 0:
    print("Analyzing non-optimal shortcuts...")
    
    # Check if shortcut cost is higher or lower than true
    non_optimal['status'] = np.where(
        non_optimal['cost'] > non_optimal['true_cost'],
        'SUBOPTIMAL (cost too high)',
        'BETTER THAN TRUE (suspicious)'
    )
    print(non_optimal['status'].value_counts())
else:
    print("No non-optimal shortcuts to analyze.")

No non-optimal shortcuts to analyze.
